<a href="https://colab.research.google.com/github/kkkkkostya/repo/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%A1%D0%9A%D0%A4%D0%9E24_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import pandas as pd
import re

# Для отрисовки графиков
import seaborn as sns
import warnings

# Для работы с файлами
import os
from os import listdir

# Основа для модели нейронной сети
from tensorflow.keras.models import Model

# Стандартные слои keras
from tensorflow.keras.layers import Input, Conv2DTranspose, concatenate, Activation, MaxPooling2D, Conv2D, BatchNormalization

# Оптимизатор Adam
from tensorflow.keras.optimizers import Adam

# Дополнительные утилиты keras
from tensorflow.keras import utils

# Инструменты для построения графиков
import matplotlib.pyplot as plt
%matplotlib inline

# Инструменты для работы с изображениями
from tensorflow.keras.preprocessing import image

# Инструменты для работы с массивами
import numpy as np

# Системные инструменты
import time, random, gdown, os
from os import listdir

# Дополнительные инструменты для работы с изображениями
from PIL import Image, ImageDraw

# Дополнительные инструменты визуализации
import seaborn as sns
sns.set_style('darkgrid')

In [ ]:
gdown.download('https://lodmedia.hb.bizmrg.com/case_files/1112984/train_dataset_train_data_Minprirodi.zip', None, quiet=True)

In [ ]:
# загрузка тренировочного датасета
gdown.download('https://lodmedia.hb.bizmrg.com/case_files/1112984/train_dataset_train_data_Minprirodi.zip', None, quiet=True)

'train_dataset_train_data_Minprirodi.zip'

In [ ]:
!unzip -P y6rX2qHwhMmvakeP5YJELd '/content//train_dataset_train_data_Minprirodi.zip' -d /content/

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: /content/train_data_Minprirodi/traps/30/IMG_2780.JPG  
  inflating: /content/train_data_Minprirodi/traps/30/IMG_2794.JPG  
  inflating: /content/train_data_Minprirodi/traps/30/IMG_3105.JPG  
  inflating: /content/train_data_Minprirodi/traps/30/IMG_1074.JPG  
  inflating: /content/train_data_Minprirodi/traps/30/IMG_1060.JPG  
  inflating: /content/train_data_Minprirodi/traps/30/IMG_2569.JPG  
  inflating: /content/train_data_Minprirodi/traps/30/IMG_3111.JPG  
  inflating: /content/train_data_Minprirodi/traps/30/IMG_1706.JPG  
  inflating: /content/train_data_Minprirodi/traps/30/IMG_3139.JPG  
  inflating: /content/train_data_Minprirodi/traps/30/IMG_0430.JPG  
  inflating: /content/train_data_Minprirodi/traps/30/IMG_1048.JPG  
  inflating: /content/train_data_Minprirodi/traps/30/IMG_0356.JPG  
  inflating: /content/train_data_Minprirodi/traps/30/IMG_2541.JPG  
  inflating: /content/train_data_Minprirodi/trap

In [ ]:
from PIL.ExifTags import TAGS
exif_data = Image.open('/content/train_data_Minprirodi/traps/1/IMG_0005.JPG')._getexif()

exif = {TAGS.get(tag): value for tag, value in exif_data.items() if tag in TAGS}
exif

{'ResolutionUnit': 2,
 'ExifOffset': 174,
 'Make': 'RECONYX',
 'Model': 'PC900 PROFESSIONAL\x00',
 'DateTime': '2020:07:22 07:02:55',
 'YCbCrPositioning': 2,
 'XResolution': 72.0,
 'YResolution': 72.0,
 'ExifVersion': b'0220',
 'ComponentsConfiguration': b'\x01\x02\x03\x00',
 'FlashPixVersion': b'0100',
 'DateTimeOriginal': '2020:07:22 07:02:55',
 'DateTimeDigitized': '2020:07:22 07:02:55',
 'ColorSpace': 1,
 'ExifImageWidth': 2048,
 'ISOSpeedRatings': 125,
 'ExifImageHeight': 1536,
 'Flash': 25,
 'ExposureMode': 0,
 'WhiteBalance': 1,
 'SceneCaptureType': 0,
 'ExposureTime': 0.016666666666666666,
 'MakerNote': b'\x01\xf1\x04\x00\x00\x00\x00\x00\x13 &\x02M\x00\x02\x00\x03\x00\x00\x00\x02\x007\x00\x02\x00\x07\x00\x07\x00\x16\x00\xe4\x07\x04\x00\x00\x002\x00\n\x00P\x009\x000\x000\x00H\x00G\x000\x005\x001\x008\x000\x000\x004\x004\x00\x00\x00\xa0\x00\x00\x00 \x00\x00\x00\x01\x00d\x00\xfc\x1cPC900 PROFESSIONAL\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [ ]:
from datetime import datetime, timedelta

root_folder = '/content/train_data_Minprirodi/traps'
date_format = '%Y:%m:%d %H:%M:%S'

traps = {}
path_time = {}

# Проходимся по всем папкам и под-папкам
for root, dirs, files in os.walk(root_folder):
    for dir in dirs:
      traps[dir] = []
      for files in os.listdir(os.path.join(root, dir)):
        path = os.path.join(root, dir,files)
        date = datetime.strptime(Image.open(path)._getexif()[306], date_format)
        traps[dir].append((path, date))
        path_time[dir+"/"+files] = date
      traps[dir].sort(key = lambda x: x[1])

In [ ]:
register_groups = {}

for i in traps:
  register_groups[i] = [[traps[i][0][0]] if len(traps[i])>0 else []]
  for j in range(1,len(traps[i])):
    if traps[i][j][1]-traps[i][j-1][1] < timedelta(minutes=30):
      register_groups[i][-1].append(traps[i][j][0])
    else:
      register_groups[i].append([traps[i][j][0]])

In [ ]:
result_reg = np.array([])

for i in register_groups:
  for j in register_groups[i]:
    model_out = model(register_groups[i][j])
    model_out['date'] = model_out['image_name'].apply(lambda x: str(i)+"/"+x).map(path_time)
    model_out = model_out[model_out['class_name']!='Empty']
    temp = model_out.groupby(['class_name']).agg({'count': 'max','date': ['min','max']})
    temp['name_foler'] = i
    if len(result_res) == 0:
      result_res =  temp[['name_folder','class_name','date','count']].values
    else:
      result_res = np.concatenate((result_res, temp[['name_folder','class_name','date','count']].values), axis=0)

registrations = pd.DataFrame(result_res, columns = ['name_folder','class','date_registration_start','date_registration_end','count'])

{'22': [{'/content/train_data_Minprirodi/traps/22/IMG_0062.JPG',
   '/content/train_data_Minprirodi/traps/22/IMG_0063.JPG',
   '/content/train_data_Minprirodi/traps/22/IMG_0064.JPG',
   '/content/train_data_Minprirodi/traps/22/IMG_0065.JPG',
   '/content/train_data_Minprirodi/traps/22/IMG_0066.JPG',
   '/content/train_data_Minprirodi/traps/22/IMG_0067.JPG',
   '/content/train_data_Minprirodi/traps/22/IMG_0068.JPG',
   '/content/train_data_Minprirodi/traps/22/IMG_0069.JPG',
   '/content/train_data_Minprirodi/traps/22/IMG_0070.JPG',
   '/content/train_data_Minprirodi/traps/22/IMG_0071.JPG',
   '/content/train_data_Minprirodi/traps/22/IMG_0072.JPG',
   '/content/train_data_Minprirodi/traps/22/IMG_0073.JPG',
   '/content/train_data_Minprirodi/traps/22/IMG_0074.JPG',
   '/content/train_data_Minprirodi/traps/22/IMG_0075.JPG',
   '/content/train_data_Minprirodi/traps/22/IMG_0076.JPG',
   '/content/train_data_Minprirodi/traps/22/IMG_0077.JPG',
   '/content/train_data_Minprirodi/traps/22/IMG_00